In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("HTwtApp").getOrCreate()

In [3]:
df=spark.read.csv("F:\\datasets\\weight-height.csv",header=True,inferSchema=True)
df.show()

+------+----------------+----------------+
|Gender|          Height|          Weight|
+------+----------------+----------------+
|  Male| 73.847017017515|241.893563180437|
|  Male|68.7819040458903|  162.3104725213|
|  Male|74.1101053917849|  212.7408555565|
|  Male|71.7309784033377|220.042470303077|
|  Male|69.8817958611153|206.349800623871|
|  Male|67.2530156878065|152.212155757083|
|  Male|68.7850812516616|183.927888604031|
|  Male|68.3485155115879|167.971110489509|
|  Male| 67.018949662883| 175.92944039571|
|  Male|63.4564939783664|156.399676387112|
|  Male|71.1953822829745|186.604925560358|
|  Male|71.6408051192206|213.741169489411|
|  Male|64.7663291334055|167.127461073476|
|  Male|69.2830700967204|189.446181386738|
|  Male|69.2437322298112|186.434168021239|
|  Male|67.6456197004212|172.186930058117|
|  Male|72.4183166259878|196.028506330482|
|  Male| 63.974325721061| 172.88347020878|
|  Male|69.6400598997523| 185.98395757313|
|  Male|67.9360048540095|182.426648013226|
+------+---

In [4]:
df.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)



In [15]:
#convertion of  (independent feature )string col to numeric col
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [11]:
indexer=StringIndexer(inputCol='Gender',outputCol='Gender_num')
dff=indexer.fit(df).transform(df)
dff.show()
print(dff.count())

+------+----------------+----------------+----------+
|Gender|          Height|          Weight|Gender_num|
+------+----------------+----------------+----------+
|  Male| 73.847017017515|241.893563180437|       1.0|
|  Male|68.7819040458903|  162.3104725213|       1.0|
|  Male|74.1101053917849|  212.7408555565|       1.0|
|  Male|71.7309784033377|220.042470303077|       1.0|
|  Male|69.8817958611153|206.349800623871|       1.0|
|  Male|67.2530156878065|152.212155757083|       1.0|
|  Male|68.7850812516616|183.927888604031|       1.0|
|  Male|68.3485155115879|167.971110489509|       1.0|
|  Male| 67.018949662883| 175.92944039571|       1.0|
|  Male|63.4564939783664|156.399676387112|       1.0|
|  Male|71.1953822829745|186.604925560358|       1.0|
|  Male|71.6408051192206|213.741169489411|       1.0|
|  Male|64.7663291334055|167.127461073476|       1.0|
|  Male|69.2830700967204|189.446181386738|       1.0|
|  Male|69.2437322298112|186.434168021239|       1.0|
|  Male|67.6456197004212|172

In [14]:
dff.na.drop(how='any')
print(dff.count())

10000


In [18]:
featureAss=VectorAssembler(inputCols=['Height','Gender_num'],outputCol='inputfeatures')
outputframe=featureAss.transform(dff)

In [20]:
outputframe.show(5,False)

+------+----------------+----------------+----------+----------------------+
|Gender|Height          |Weight          |Gender_num|inputfeatures         |
+------+----------------+----------------+----------+----------------------+
|Male  |73.847017017515 |241.893563180437|1.0       |[73.847017017515,1.0] |
|Male  |68.7819040458903|162.3104725213  |1.0       |[68.7819040458903,1.0]|
|Male  |74.1101053917849|212.7408555565  |1.0       |[74.1101053917849,1.0]|
|Male  |71.7309784033377|220.042470303077|1.0       |[71.7309784033377,1.0]|
|Male  |69.8817958611153|206.349800623871|1.0       |[69.8817958611153,1.0]|
+------+----------------+----------------+----------+----------------------+
only showing top 5 rows



In [21]:
finaldata=outputframe.select('inputfeatures','Weight')

In [22]:
finaldata.printSchema()

root
 |-- inputfeatures: vector (nullable = true)
 |-- Weight: double (nullable = true)



In [24]:
finaldata.show(5,False)

+----------------------+----------------+
|inputfeatures         |Weight          |
+----------------------+----------------+
|[73.847017017515,1.0] |241.893563180437|
|[68.7819040458903,1.0]|162.3104725213  |
|[74.1101053917849,1.0]|212.7408555565  |
|[71.7309784033377,1.0]|220.042470303077|
|[69.8817958611153,1.0]|206.349800623871|
+----------------------+----------------+
only showing top 5 rows



In [25]:
train,test=finaldata.randomSplit([.70,.30])

In [30]:
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(featuresCol='inputfeatures',labelCol='Weight')
lrt=lr.fit(train)

In [33]:
print(lrt.coefficients)
print(lrt.intercept)

[5.997987408838811,19.134739417155952]
-246.18523982830982


In [36]:
res=lrt.evaluate(test)
res.predictions.show(5,False)

+----------------------+----------------+-----------------+
|inputfeatures         |Weight          |prediction       |
+----------------------+----------------+-----------------+
|[55.9791978761431,0.0]|85.4175336162677|89.57728418969276|
|[56.1672991862273,0.0]|77.8985592718359|90.70551347916393|
|[56.5488430793485,0.0]|90.8475893808927|92.99400894602422|
|[56.6781404906408,0.0]|97.2699666809593|93.76953319095088|
|[56.7576036320284,0.0]|88.8848531761247|94.24615211246049|
+----------------------+----------------+-----------------+
only showing top 5 rows



In [39]:
print("r2",res.r2)
print("RMSE",res.rootMeanSquaredError)
print("MAE",res.meanAbsoluteError)

r2 0.9002641316976288
RMSE 9.999081010863307
MAE 7.957034276057769


In [40]:
#dftip=sns.load_dataset('tips')
tip--predict bill
dependecny of tip on total bill

NameError: name 'sns' is not defined